In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout
from keras.layers.convolutional import MaxPooling2D, Conv2D

In [3]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = train.drop('label',axis=1)  #these will be used only for training
y_train = train['label']

In [7]:
X_train.shape

(42000, 784)

In [8]:
y_train.shape

(42000,)

In [9]:
y_train.unique()  #we have 10 unique numbers

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6])

In [10]:
y_train.value_counts()  #they seem to be more or less balanced

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [11]:
train.isna().sum().sum() #we seem to have no missing values (np.nan)

0

In [12]:
X_train = X_train/255.0 #(dividing by 255 to change the scale from 0 to 1, dividing by float helps in faster computation)
test = test/255

In [13]:
# images have 3 dimensions so we need to reshape them
X_train = X_train.values.reshape(-1,28,28,1) # -1 says keep the number of data as it it 28*28 is height* width 1 is grayscale
                                       # if we have coloured we should use 3
test = test.values.reshape(-1,28,28,1)

In [14]:
from keras.utils import np_utils

In [15]:
y_train = np_utils.to_categorical(y_train, num_classes = 10)  #to label encode them

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=99)

In [17]:
#adding convolution and max pool layers along with dropout probability=0.5 so the networks learn better
model = Sequential()

model.add(Conv2D(64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Conv2D(128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [18]:
model.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [19]:
y_valid


array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [20]:
model.fit(X_train, y_train, batch_size = 20, epochs = 10,validation_data = (X_valid, y_valid), verbose = 2)

Epoch 1/10
1680/1680 - 67s - loss: 0.2703 - accuracy: 0.9128 - val_loss: 0.0644 - val_accuracy: 0.9814
Epoch 2/10
1680/1680 - 65s - loss: 0.1089 - accuracy: 0.9664 - val_loss: 0.0509 - val_accuracy: 0.9831
Epoch 3/10
1680/1680 - 65s - loss: 0.0900 - accuracy: 0.9725 - val_loss: 0.0420 - val_accuracy: 0.9868
Epoch 4/10
1680/1680 - 66s - loss: 0.0784 - accuracy: 0.9760 - val_loss: 0.0373 - val_accuracy: 0.9883
Epoch 5/10
1680/1680 - 66s - loss: 0.0723 - accuracy: 0.9781 - val_loss: 0.0396 - val_accuracy: 0.9882
Epoch 6/10
1680/1680 - 65s - loss: 0.0640 - accuracy: 0.9801 - val_loss: 0.0365 - val_accuracy: 0.9885
Epoch 7/10
1680/1680 - 65s - loss: 0.0609 - accuracy: 0.9813 - val_loss: 0.0337 - val_accuracy: 0.9894
Epoch 8/10
1680/1680 - 65s - loss: 0.0578 - accuracy: 0.9825 - val_loss: 0.0341 - val_accuracy: 0.9894
Epoch 9/10
1680/1680 - 66s - loss: 0.0557 - accuracy: 0.9828 - val_loss: 0.0368 - val_accuracy: 0.9883
Epoch 10/10
1680/1680 - 66s - loss: 0.0561 - accuracy: 0.9833 - val_loss:

In [21]:
#thank you so much for stopping by <3